In [2]:
## Load the LabOne API and other necessary packages
from zhinst.toolkit import Session,  Waveforms
from zhinst.toolkit import CommandTable
import numpy as np

device_id='DEV9175'
server_host = 'localhost'

### connect to data server
session = Session(server_host)
### connect to device
device = session.connect_device(device_id)

seqc_program = """\

const LENGTH = 240000;

wave w1 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker
wave w2 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker

assignWaveIndex(w1,w2,10);                 // Create a wave table entry with placeholder waveform              
                                         // routed to output 1, with index 10

// execute the command table entry
executeTableEntry(0);

while(true){
  waitDigTrigger(1);
  playWave(w1,w2);

 waitWave();
};
"""
AWG_INDEX = 0 #use channel 1&2 for 2*2
awg = device.awgs[AWG_INDEX]
WAVE_INDEX = 0
TABLE_INDEX = 0

# load the sequence
awg.load_sequencer_program(seqc_program)


# initialize the command table
ct_schema = awg.commandtable.load_validation_schema()
ct = CommandTable(ct_schema)

# Wavefrom eith amplitude and phase settings
ct.table[TABLE_INDEX].waveform.index = WAVE_INDEX
ct.table[TABLE_INDEX].amplitude0.value = 1
ct.table[TABLE_INDEX].amplitude1.value = 1


##Generate a waveform and marker
Fs = 2400 # in MHz
nu = 100 # microwave frequency in MHz
t_mw = 100 # MW duration in us
A = 0.5

LENGTH = t_mw * Fs
# number of points
wave1 = A*np.sin(np.linspace(0, 2*np.pi*nu*t_mw, LENGTH))
wave2 = A*np.cos(np.linspace(0, 2*np.pi*nu*t_mw, LENGTH))
# marker = np.concatenate([np.ones(32), np.zeros(LENGTH-32)]).astype(int)

## Upload waveforms

print(seqc_program)

waveforms = Waveforms()
waveforms[10] = (wave1,wave2,None) # I-component (output channel 1), Q-component (output channel 2), marker
awg.commandtable.upload_to_device(ct)
awg.write_to_waveform_memory(waveforms)
awg.enable_sequencer(single=True)
# awg.wait_done()


const LENGTH = 240000;

wave w1 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker
wave w2 = placeholder(LENGTH, false, false); // Create a waveform of size LENGTH, with one marker

assignWaveIndex(w1,w2,10);                 // Create a wave table entry with placeholder waveform              
                                         // routed to output 1, with index 10

// execute the command table entry
executeTableEntry(0);

while(true){
  waitDigTrigger(1);
  playWave(w1,w2);

 waitWave();
};

